In [1]:
from ortools.sat.python import cp_model
from scipy.spatial.distance import pdist, squareform

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import time
from itertools import cycle
import pandas as pd

In [2]:
N = 100
M = 10
max_timeslots = 20

In [3]:
data = np.array(pd.read_csv('data.csv'))
data = data[0:1000]
list1 = data.T[0]
list2 = data.T[1]
list1
list2

array([  5,   7,   9,  10,  11,  12,  13,  14,  15,  16,  18,  19,  22,
        25,  27,  28,  30,  31,  32,  33,  35,  36,  37,  39,  40,  41,
        42,  45,  46,  47,  51,  53,  54,  55,  58,  59,  60,  62,  63,
        65,  69,  70,  71,  72,  75,  81,  84,  87,  88,  89,  91,  93,
        95,  96,  98,   3,   5,   6,   7,  10,  12,  13,  16,  17,  19,
        20,  21,  22,  23,  25,  26,  27,  28,  30,  31,  32,  33,  34,
        37,  40,  41,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        54,  56,  57,  58,  59,  62,  63,  66,  70,  81,  82,  83,  84,
        85,  86,  87,  88,  96, 100,   4,   5,   7,  14,  15,  16,  17,
        18,  19,  23,  24,  25,  27,  32,  35,  39,  44,  46,  47,  48,
        52,  55,  58,  60,  61,  62,  63,  64,  65,  68,  70,  75,  76,
        77,  78,  80,  81,  83,  85,  88,  89,  92,  94, 100,   5,   7,
        10,  11,  14,  16,  17,  19,  21,  24,  28,  29,  30,  31,  32,
        33,  37,  40,  43,  46,  48,  49,  50,  52,  53,  55,  5

In [4]:
tuples_list = []
for i in range(len(list1)):
    tuples_list.append(tuple((list1[i],list2[i])))
tuples_list

[(1, 5),
 (1, 7),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (1, 16),
 (1, 18),
 (1, 19),
 (1, 22),
 (1, 25),
 (1, 27),
 (1, 28),
 (1, 30),
 (1, 31),
 (1, 32),
 (1, 33),
 (1, 35),
 (1, 36),
 (1, 37),
 (1, 39),
 (1, 40),
 (1, 41),
 (1, 42),
 (1, 45),
 (1, 46),
 (1, 47),
 (1, 51),
 (1, 53),
 (1, 54),
 (1, 55),
 (1, 58),
 (1, 59),
 (1, 60),
 (1, 62),
 (1, 63),
 (1, 65),
 (1, 69),
 (1, 70),
 (1, 71),
 (1, 72),
 (1, 75),
 (1, 81),
 (1, 84),
 (1, 87),
 (1, 88),
 (1, 89),
 (1, 91),
 (1, 93),
 (1, 95),
 (1, 96),
 (1, 98),
 (2, 3),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 10),
 (2, 12),
 (2, 13),
 (2, 16),
 (2, 17),
 (2, 19),
 (2, 20),
 (2, 21),
 (2, 22),
 (2, 23),
 (2, 25),
 (2, 26),
 (2, 27),
 (2, 28),
 (2, 30),
 (2, 31),
 (2, 32),
 (2, 33),
 (2, 34),
 (2, 37),
 (2, 40),
 (2, 41),
 (2, 43),
 (2, 44),
 (2, 45),
 (2, 46),
 (2, 47),
 (2, 48),
 (2, 49),
 (2, 50),
 (2, 51),
 (2, 52),
 (2, 54),
 (2, 56),
 (2, 57),
 (2, 58),
 (2, 59),
 (2, 62),
 (2, 63),
 (2, 66),
 (2, 70),
 (2, 81

## Sets

In [5]:
N_set = [i for i in range(1,N+1)]
M_set = [i for i in range(1,M+1)]
T_set = [i for i in range(1,max_timeslots+1)]

## Model

In [6]:
model = cp_model.CpModel()

In [7]:
X_vars = {(i,j,t): model.NewBoolVar(f"{i}_{j}_{t}") 
          for i in N_set for j in M_set for t in T_set}

Y_vars = {(j,t): model.NewBoolVar(f"{j}_{t}") 
          for j in M_set for t in T_set}

Z_vars = {(j): model.NewBoolVar(f"{j}") 
          for j in M_set}

## Constraints

In [8]:
# each even to be allocated once
for i in N_set:
    model.Add(sum(X_vars[i,j,t] for j in M_set for t in T_set) == 1)

# add event constraints given in question
for idx,rm in enumerate(list1):
    for j in M_set:
        for t in T_set:
                model.Add(X_vars[list1[idx],j,t] + X_vars[list2[idx],j,t] <= 1)

# each room to be allocated
for i in N_set:
    for j in M_set:
        for t in T_set:
            # model.Add(X_vars[i,j,t] == Y_vars[j,t])
            model.Add(X_vars[i,j,t] <= Z_vars[j])
        # model.Add(X_vars[i,j] == Y_vars[j]).OnlyEnforceIf(X_vars[i,j])

#each timeslot to be allocated
for j in M_set:
    for t in T_set:
        model.Add(sum(X_vars[i,j,t] for i in N_set) <= 1)

'''
for i in N_set:
    for j in M_set:
        model.AddBoolOr(X_vars[i,j], Y_vars[j])
'''

'\nfor i in N_set:\n    for j in M_set:\n        model.AddBoolOr(X_vars[i,j], Y_vars[j])\n'

## Objective

In [9]:
obj = sum(Z_vars[j] for j in M_set) 
model.Minimize(obj)

In [10]:
print(model.ModelStats())

optimization model '': (model_fingerprint: 0x16a3aff6a6324a8b)
#Variables: 20'210 (#bools: 10 in objective)
  - 20'210 Booleans in [0,1]
#kLinear2: 220'000
#kLinearN: 300 (#terms: 40'000)


In [ ]:
# Solve
THREADS = 4
TIME_LIMIT = 300
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = TIME_LIMIT
solver.parameters.log_search_progress = True
solver.parameters.num_workers = THREADS
solver.log_callback = print
status = solver.Solve(model)


Starting CP-SAT solver v9.10.4067
Parameters: max_time_in_seconds: 300 log_search_progress: true num_workers: 4

Initial optimization model '': (model_fingerprint: 0x16a3aff6a6324a8b)
#Variables: 20'210 (#bools: 10 in objective)
  - 20'210 Booleans in [0,1]
#kLinear2: 220'000
#kLinearN: 300 (#terms: 40'000)

Starting presolve at 0.20s
  8.89e-02s  0.00e+00d  [DetectDominanceRelations] 
  1.80e+00s  0.00e+00d  [operations_research::sat::CpModelPresolver::PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  4.39e-03s  0.00e+00d  [operations_research::sat::CpModelPresolver::ExtractEncodingFromLinear] #potential_supersets=300 
[Symmetry] Graph for symmetry has 60'530 nodes and 520'020 arcs.
[Symmetry] Symmetry computation done. time: 0.625933 dtime: 0.391899
[Symmetry] #generators: 190, average support size: 380.095
[Symmetry] 101 orbits with sizes: 200,200,200,200,200,200,200,200,200,200,...
[Symmetry] Num fixable by binary propagation in orbit: 199 / 200
[Symmetry] Found orbito

In [ ]:
print (solver.response_stats())

CpSolverResponse summary:
status: FEASIBLE
objective: 7
best_bound: 1
integers: 20
booleans: 26627
conflicts: 49425
branches: 1044904
propagations: 42728089
integer_propagations: 300331
restarts: 196060
lp_iterations: 0
walltime: 60.9225
usertime: 60.9225
deterministic_time: 352.744
gap_integral: 626.224
solution_fingerprint: 0xd2dbbc66efdcb7ee



In [ ]:
if status == cp_model.OPTIMAL:
    print("Optimal")
elif status == cp_model.FEASIBLE:
    print("Feasible")
else:
    print ("No solution")

print(solver.ObjectiveValue())

print('Rooms used:', sum(
    solver.Value(Z_vars[j]) 
    for j in M_set
))

Feasible
7.0
Rooms used: 7


In [ ]:
for t in T_set:
    print ("Time slot:", t)
    time_dict = {}
    for j in M_set:
        # print ("Room Number:", j)
        for i in N_set:
            if solver.value(X_vars[i,j,t])==1:
                time_dict[j] = i
    print (time_dict)

Time slot: 1
{3: 34, 4: 50, 5: 66, 6: 82, 10: 12, 20: 2}
Time slot: 2
{3: 35, 4: 51, 5: 67, 6: 83, 10: 89, 20: 100}
Time slot: 3
{3: 36, 4: 52, 5: 68, 6: 28, 10: 11, 20: 99}
Time slot: 4
{3: 37, 4: 53, 5: 69, 6: 27, 10: 43, 20: 98}
Time slot: 5
{3: 38, 4: 54, 5: 70, 6: 26, 10: 10, 20: 21}
Time slot: 6
{3: 39, 4: 55, 5: 71, 6: 25, 10: 9, 20: 96}
Time slot: 7
{3: 40, 4: 56, 5: 72, 6: 24, 10: 97, 16: 92, 20: 88}
Time slot: 8
{3: 41, 4: 57, 5: 73, 6: 20, 16: 84, 20: 94}
Time slot: 9
{3: 33, 4: 58, 5: 74, 6: 19, 10: 86, 20: 22}
Time slot: 10
{3: 32, 4: 59, 5: 75, 6: 18, 10: 91, 16: 87, 20: 8}
Time slot: 11
{3: 31, 4: 60, 5: 76, 6: 17, 10: 93, 16: 42, 20: 5}
Time slot: 12
{3: 30, 4: 61, 5: 77, 6: 16, 16: 95, 20: 4}
Time slot: 13
{3: 29, 4: 62, 5: 78, 6: 15, 10: 85, 20: 7}
Time slot: 14
{3: 47, 4: 63, 5: 79, 6: 14, 10: 90, 16: 45, 20: 3}
Time slot: 15
{3: 48, 4: 64, 5: 80, 6: 46, 16: 44, 20: 6}
Time slot: 16
{3: 49, 4: 65, 5: 81, 6: 13, 10: 23, 20: 1}


In [ ]:
for j in M_set:
    if solver.value(Z_vars[j])==1:
        print (j)

3
4
5
6
10
16
20


In [ ]:
for j in M_set:
    print ("Room Number:", j) 
    for t in T_set:
        print ("Time slot:", t)
        for i in N_set:
            if solver.value(X_vars[i,j,t])==1:
                print (i)
  

Room Number: 1
Time slot: 1
Time slot: 2
Time slot: 3
Time slot: 4
Time slot: 5
Time slot: 6
Time slot: 7
Time slot: 8
Time slot: 9
Time slot: 10
Time slot: 11
Time slot: 12
Time slot: 13
Time slot: 14
Time slot: 15
Time slot: 16
Room Number: 2
Time slot: 1
Time slot: 2
Time slot: 3
Time slot: 4
Time slot: 5
Time slot: 6
Time slot: 7
Time slot: 8
Time slot: 9
Time slot: 10
Time slot: 11
Time slot: 12
Time slot: 13
Time slot: 14
Time slot: 15
Time slot: 16
Room Number: 3
Time slot: 1
34
Time slot: 2
35
Time slot: 3
36
Time slot: 4
37
Time slot: 5
38
Time slot: 6
39
Time slot: 7
40
Time slot: 8
41
Time slot: 9
33
Time slot: 10
32
Time slot: 11
31
Time slot: 12
30
Time slot: 13
29
Time slot: 14
47
Time slot: 15
48
Time slot: 16
49
Room Number: 4
Time slot: 1
50
Time slot: 2
51
Time slot: 3
52
Time slot: 4
53
Time slot: 5
54
Time slot: 6
55
Time slot: 7
56
Time slot: 8
57
Time slot: 9
58
Time slot: 10
59
Time slot: 11
60
Time slot: 12
61
Time slot: 13
62
Time slot: 14
63
Time slot: 15
64
T

In [ ]:
# Events per room

for j in M_set:
    events = sum(solver.value(X_vars[i,j,t]) for i in N_set for t in T_set)
    if events >0:
        print ("Room Number:", j)
        print ("Number of events:", events, '\n')

Room Number: 3
Number of events: 16 

Room Number: 4
Number of events: 16 

Room Number: 5
Number of events: 16 

Room Number: 6
Number of events: 16 

Room Number: 10
Number of events: 13 

Room Number: 16
Number of events: 7 

Room Number: 20
Number of events: 16 

